# Neural Bag of Words
<img src="imgs/01.nbow-intro.png" width="800" height="800" />

## Why NBoW?
- **A classic “Hello World” task for NLP.**  
  Neural Bag-of-Words (NBoW) provides a simple yet effective baseline model for text classification, making it an ideal starting point for understanding core NLP concepts.

- **Introduction to sentiment analysis.**  
  In this tutorial, we build a machine learning model for *sentiment analysis*, a fundamental subtask of text classification.  
  The objective is to determine whether a given sentence or review expresses **positive** or **negative** sentiment.

- **Real-world dataset: IMDb movie reviews.**  
  The model is trained and evaluated on the widely used Internet Movie Database (IMDb) dataset, which contains labeled movie reviews.

- **Foundational NLP concepts.**  
  Along the way, essential techniques are introduced and explained, including:
  - Tokenization  
  - Vocabulary construction  
  - Numerical encoding of text  
  - Word embeddings  

## Global and imports settings

In [1]:
import datasets
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os
import requests
import zipfile
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import collections

seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)

## Dataset Preparation

- **Load the dataset** using the `datasets` library.
- **Split the dataset** into training and test sets.
- **Inspect the data structure:**
  - Each sample consists of:
    - `text`: the movie review
    - `label`: the sentiment label  
      - `0` → negative review  
      - `1` → positive review

In [2]:
train_data_raw, test_data_raw = datasets.load_dataset("imdb", split=["train", "test"])
train_data_raw, test_data_raw

(Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }))

In [3]:
train_data_raw.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [4]:
train_data_raw[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## Building input tokenizer
* Tokenizers split unstructured raw text into manageable units (words, subwords, or characters) that models can understand.
* word_tokenize is a function from Natural Language Toolkit's (NLTK) tokenize module that splits text into individual words and punctuation tokens, e.g., words, commas, periods for further NLP processing.

In [5]:
nltk.download('punkt_tab')
def get_tokens(text):
    return word_tokenize(text.lower())  # Tokenize and lowercase

text = "Hello, Mr. Smith! Isn't NLP cool?"
print("Tokenizer: ", word_tokenize(text))  
print("Split: ", text.split()) 

Tokenizer:  ['Hello', ',', 'Mr.', 'Smith', '!', 'Is', "n't", 'NLP', 'cool', '?']
Split:  ['Hello,', 'Mr.', 'Smith!', "Isn't", 'NLP', 'cool?']


[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Applying Tokenizer: Mapping input text to tokens
* Preprocess data: apply tokenizer to training and test data

In [6]:
max_length = 256

def tokenize_input(input, max_length):
    tokens = get_tokens(input["text"])[:max_length] # truncate text if too long
    return {"tokens": tokens}
    
train_data_tok = train_data_raw.map(
    tokenize_input, fn_kwargs={"max_length": max_length}
)
test_data_tok = test_data_raw.map(
    tokenize_input, fn_kwargs={"max_length": max_length}
)

* **tokenize_input** is a helper function that performs tokenization of text data from the dataset using the specified tokenizer. Its primary purposes are:
  * Tokenize Text: Converts raw text (from the dataset) into tokens
  * Truncating longer sequences to fit within max_length

In [7]:
train_data_tok

Dataset({
    features: ['text', 'label', 'tokens'],
    num_rows: 25000
})

In [8]:
train_data_tok.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [9]:
train_data_tok[0]["tokens"][:25]

['i',
 'rented',
 'i',
 'am',
 'curious-yellow',
 'from',
 'my',
 'video',
 'store',
 'because',
 'of',
 'all',
 'the',
 'controversy',
 'that',
 'surrounded',
 'it',
 'when',
 'it',
 'was',
 'first',
 'released',
 'in',
 '1967.',
 'i']

## Building Vocabulary

* *Numerical Foundation*: Neural networks process numbers (tensors), not raw text.
* *NBow Vocabulary Mapping*: Maps each unique token to a unique integer index.
* *Corpus Analysis*: Builds the vocabulary by calculating token frequencies across the training set.
* *Generalization via Filtering*: Removes rare tokens (below min_freq) to reduce noise and prevent overfitting.
* *Special Tokens*: Integrates reserved tokens for <UNK> (out-of-vocabulary) and <PAD> (sequence length normalization). 

In [10]:
# Building Vocabulary: Mapping word → unique ID
all_tokens = [token for example in train_data_tok for token in example["tokens"]]
word_counts = Counter(all_tokens)  # Count word occurrences

# Assign unique IDs (starting from 2 to reserve 0 for PAD, 1 for UNK)
min_freq = 5

word_counts_filtered = {x: count for x, count in word_counts.items() if count >= min_freq}

vocab = {word: idx + 2 for idx, word in enumerate(word_counts_filtered) }
pad_token_index = 0
unknown_token_index = 1
vocab["[PAD]"] = pad_token_index  # Padding token
vocab["[UNK]"] = unknown_token_index  # Unknown token

# Print vocabulary size
print(f"Vocabulary Size: {len(vocab)}")

# Print first 10 words in vocabulary
print(list(vocab.keys())[500:510])

Vocabulary Size: 25607
['themselves', 'jr.', 'how', 'filmmakers', 'hard', 'seems', 'awfully', 'sloppy', 'saw', 'glimpse']


## Apply vocablurary to text tokens
* create a numerical vector from tokenized text

In [11]:
def numericalize_dataset(example, vocab):
    # Use UNK for unknown words
    ids = [vocab.get(token, unknown_token_index) for token in example["tokens"]] 
    return {"ids": ids}

In [12]:
train_data_num = train_data_tok.map(numericalize_dataset, fn_kwargs={"vocab": vocab})
test_data_num = test_data_tok.map(numericalize_dataset, fn_kwargs={"vocab": vocab})

In [13]:
# Print the first example before and after tokenization
original_text = train_data_num[0]["text"]  # Get original text
tokenized_ids = train_data_num[0]["ids"]  # Get tokenized input IDs

vocab_inv = {v: k for k, v in vocab.items()}  # Invert mapping
tokenized_text = [vocab_inv.get(token, "[UNK]") for token in tokenized_ids]
print(f"Original Text:\n{original_text}\n")
print(f"Tokens:\n{tokenized_text}\n")
print(f"Token IDS:\n{tokenized_ids}\n")

Original Text:
I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and f

In [14]:
train_data_tensor = train_data_num.with_format(type="torch", columns=["ids", "label"])
test_data_tensor = test_data_num.with_format(type="torch", columns=["ids", "label"])

## Building dataloader

* The get_collate_fn function defines a custom collation function for PyTorch's DataLoader that processes batches of variable-length sequences. 
* Handles batches where input sequences have different lengths by:
  * Padding sequences to match the longest sequence in the batch
  * Properly stacking labels
```
[
  {"ids": tensor([5, 9, 2]),     "label": tensor(1)},
  {"ids": tensor([3, 4]),        "label": tensor(0)},
  {"ids": tensor([7, 8, 6, 1]),  "label": tensor(1)}
]
```
-->
```
{
  "ids": tensor([
      [5, 9, 2, 0],
      [3, 4, 0, 0],
      [7, 8, 6, 1]
  ]),
  "label": tensor([1, 0, 1])
}
```

In [15]:
def get_collate_fn(pad_token_index):
    def collate_fn(batch):
        ids = [item["ids"] for item in batch]
        labels = [item["label"] for item in batch]

        ids = nn.utils.rnn.pad_sequence(
            ids, batch_first=True, padding_value=pad_token_index
        )
        labels = torch.stack(labels)

        return {"ids": ids, "label": labels}

    return collate_fn

* get_data_loader creates a PyTorch DataLoader configured with a custom collation function for handling variable-length sequences. It wraps PyTorch's standard DataLoader while incorporating the padding logic from the get_collate_fn function.

In [16]:
def get_data_loader(dataset, batch_size, pad_token_index, shuffle=False):
    collate_fn = get_collate_fn(pad_token_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [17]:
batch_size = 512

train_data_loader = get_data_loader(train_data_tensor, batch_size, pad_token_index, shuffle=True)
test_data_loader = get_data_loader(test_data_tensor, batch_size, pad_token_index)

## Building the Network with PyTorch

<img src="imgs/nbow_model.png" width="800" height="800" />
A simple neural network for text classification that treats input text as an unordered set of words (bag-of-words representation).

In [18]:
class NBoW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_token_index):
        super().__init__()
        # Embedding layer:
        # Maps each token ID -> embedding vector
        # padding_idx ensures that embeddings for PAD tokens are always zero
        self.embedding = nn.Embedding(
            vocab_size, 
            embedding_dim, 
            padding_idx=pad_token_index
        )
        
        # Fully-connected (linear) layer:
        # Maps embedding_dim -> output_dim (e.g., number of classes)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, ids):
        # Convert token IDs into embeddings
        # Result shape: [batch_size, sequence_length, embedding_dim]
        embedded = self.embedding(ids)
        
        # Pool across the sequence dimension using mean
        # This collapses variable-length sequences into a single vector
        # Result shape: [batch_size, embedding_dim]
        pooled = embedded.mean(dim=1)
        
        # Apply linear layer to get class scores (logits)
        prediction = self.fc(pooled)
        
        # Result shape: [batch_size, output_dim]
        return prediction

### Embedding Layer
* Maps token IDs → dense vectors

### Mean Pooling
* Averages word embeddings along sequence dimension
* Converts [batch_size, seq_len, emb_dim] → [batch_size, emb_dim]
* Captures overall document semantics

### Linear Classifier
* Maps pooled embedding → class logits
* Output_dim: Number of classes (e.g., 2 for binary classification)

In [19]:
vocab_size = len(vocab)
embedding_dim = 300
output_dim = len(train_data_tensor.unique("label"))

model = NBoW(vocab_size, embedding_dim, output_dim, pad_token_index)

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 7,682,702 trainable parameters


### Download pre-trained embedding
* Code just downloads archive and unarchives desired dimensionality.
* Pretrained embedding capture simularities, e.g. if words are similar then cosine is -> 1
```
             king    france  computer
king     1.000000  0.345671  0.123456
queen    0.856734  0.234567  0.098765
prince   0.823456  0.287654  0.087654
...          ...       ...       ...
france   0.345671  1.000000  0.056789
germany  0.301234  0.912345  0.043210
paris    0.287654  0.876543  0.065432
...          ...       ...       ...
computer 0.123456  0.056789  1.000000
software 0.098765  0.043210  0.876543
keyboard 0.087654  0.032109  0.812345
````

In [21]:
def download_glove(dim=100, save_dir="glove"):
    assert dim in [50, 100, 200, 300], "Invalid dimension. Must be one of [50, 100, 200, 300]"
    url = "http://nlp.stanford.edu/data/glove.6B.zip"
    zip_path = os.path.join(save_dir, "glove.6B.zip")
    target_file = f"glove.6B.{dim}d.txt"
    target_path = os.path.join(save_dir, target_file)

    os.makedirs(save_dir, exist_ok=True)

    if not os.path.exists(target_path):
        if not os.path.exists(zip_path):
            print("Downloading GloVe embeddings...")
            response = requests.get(url, stream=True)
            total_size = int(response.headers.get('content-length', 0))
            with open(zip_path, "wb") as f, tqdm(
                desc=zip_path,
                total=total_size,
                unit="B",
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    f.write(data)
                    bar.update(len(data))

        print("Extracting...")
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extract(target_file, path=save_dir)

    print(f"GloVe {dim}d embedding ready at: {target_path}")
    return target_path

# Example usage:
glove_path = download_glove(dim=embedding_dim)

GloVe 300d embedding ready at: glove/glove.6B.300d.txt


### Load embedding matrix into the model

In [22]:
# Convert numerical token IDs back into words (tokens).
# vocab = { "[PAD]": 0,"[UNK]": 1, "the": 2, "cat": 3, "sat": 4 }
# itos = {0: "[PAD]", 1: "[UNK]", 2: "the", 3: "cat", 4: "sat" }
itos = {idx: token for token, idx in vocab.items()}

# Load GloVe embeddings from file system
def load_glove_embeddings(filepath):
    embeddings = {}
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]  # First word is the token
            vector = np.asarray(values[1:], dtype="float32")  # Remaining values are the embedding
            embeddings[word] = vector
    return embeddings

# Path to GloVe file (adjust the path to your local file)
# glove_path = "glove.6B.300d.txt"  # Use 300-dimensional GloVe embeddings
glove_embeddings = load_glove_embeddings(glove_path)

avg_glove_embedding = np.mean(np.array(list(glove_embeddings.values())), axis=0)
# Create an embedding matrix for the vocabulary
embedding_matrix = np.zeros((len(itos), embedding_dim))

for idx, token in enumerate(itos):
    if token in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[token]
    else:
        embedding_matrix[idx] = avg_glove_embedding  # init for unknown tokens

# Convert embedding matrix to PyTorch tensor
embedding_matrix = torch.tensor(embedding_matrix, dtype=torch.float32)
print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: torch.Size([25607, 300])


In [23]:
# Convert the embedding matrix to a PyTorch tensor
embedding_matrix = embedding_matrix.clone().detach().float()

# Assign to the model's embedding layer
model.embedding.weight.data.copy_(embedding_matrix)
print("GloVe embeddings loaded into the model.")

GloVe embeddings loaded into the model.


## Building Training Pipeline

In [24]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = criterion.to(device)

Adaptive Learning Rates (ADAM):
* Adam automatically adjusts learning rates for each parameter (using momentum and adaptive scaling), which:
* Works well with sparse gradients (common in NLP)
* Eliminates need for manual learning rate scheduling
* Outperforms SGD for most NLP tasks

Output Layer Behavior:
* The final layer produces raw logits (unnormalized scores)
* This is the required input format for nn.CrossEntropyLoss
* The loss function internally applies LogSoftmax + NLLLoss
```
# Equivalent to:
log_probs = F.log_softmax(model(inputs), dim=1)
loss = F.nll_loss(log_probs, labels)
```

Standard Practice:
* PyTorch classification models typically output raw logits
* This pattern is used in official tutorials and production code
* Simplifies the model architecture

### Training step

* *tqdm*: use tqdm to display a real-time progress bar during training, which provides valuable feedback about training progress, estimated time remaining, and batch processing speed. While you can use enumerate, tqdm offers significant usability advantages.

In [25]:
def get_accuracy(prediction, label):
    return (prediction.argmax(dim=-1) == label).float().mean()

In [26]:
def train(data_loader, model, criterion, optimizer, device):
    model.train()
    losses, accs = [], []

    for batch in tqdm(data_loader, desc="training..."):
        ids = batch["ids"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        preds = model(ids)
        loss = criterion(preds, labels)
        acc = get_accuracy(preds, labels)

        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        accs.append(acc.item())

    return np.mean(losses), np.mean(accs)

### Test step

In [27]:
def evaluate(dl, model, loss_fn, device):
    model.eval()
    losses, accs = [], []

    with torch.no_grad():
        for b in tqdm(dl, desc="evaluating..."):
            x, y = b["ids"].to(device), b["label"].to(device)
            p = model(x)
            l = loss_fn(p, y)
            losses.append(l.item())
            accs.append(get_accuracy(p, y).item())

    return np.mean(losses), np.mean(accs)


## Model in the loop

In [ ]:
n_epochs = 10
metrics = collections.defaultdict(list)

for epoch in range(n_epochs):
    train_loss, train_acc = train(train_data_loader, model, criterion, optimizer, device)
    test_loss, test_acc = evaluate(test_data_loader, model, criterion, device)

    metrics["train_losses"].append(train_loss)
    metrics["train_accs"].append(train_acc)
    metrics["test_losses"].append(test_loss)
    metrics["test_accs"].append(test_acc)

    print(
        f"Epoch {epoch} | "
        f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f} | "
        f"Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.3f}"
    )
torch.save(model.state_dict(), "nbow.pt")

evaluating...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:04<00:00, 11.78it/s]


Epoch 0 | Train Loss: 0.685, Train Acc: 0.572 | Test Loss: 0.672, Test Acc: 0.630


evaluating...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:04<00:00, 12.17it/s]


Epoch 1 | Train Loss: 0.636, Train Acc: 0.749 | Test Loss: 0.606, Test Acc: 0.744


evaluating...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:04<00:00, 12.11it/s]


...

evaluating...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:04<00:00, 12.04it/s]


Epoch 3 | Train Loss: 0.453, Train Acc: 0.838 | Test Loss: 0.458, Test Acc: 0.819


evaluating...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:04<00:00, 11.95it/s]


Epoch 4 | Train Loss: 0.385, Train Acc: 0.866 | Test Loss: 0.415, Test Acc: 0.836


evaluating...:  45%|████████████████████████████████████████████                                                      | 22/49 [00:01<00:02, 12.06it/s]

### Performance during training

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(metrics["train_losses"], label="train loss")
ax.plot(metrics["test_losses"], label="valid loss")
ax.set_xlabel("epoch")
ax.set_ylabel("loss")
ax.set_xticks(range(n_epochs))
ax.legend()
ax.grid()

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(metrics["train_accs"], label="train accuracy")
ax.plot(metrics["test_accs"], label="test accuracy")
ax.set_xlabel("epoch")
ax.set_ylabel("loss")
ax.set_xticks(range(n_epochs))
ax.legend()
ax.grid()

## Usage of trained model

In [ ]:
model.load_state_dict(torch.load("nbow.pt", weights_only=True))

test_loss, test_acc = evaluate(test_data_loader, model, criterion, device)

In [ ]:
print(f"test_loss: {test_loss:.3f}, test_acc: {test_acc:.3f}")

In [ ]:
def predict_sentiment(text, model, vocab, device):
    """
    Predict sentiment (positive/negative) for a single text string.

    Returns:
        predicted_class (int): 0 = negative, 1 = positive
        predicted_probability (float): confidence of the prediction
    """

    # Put model in evaluation mode
    model.eval()

    # Tokenize input text
    tokens = get_tokens(text)

    # Convert tokens to vocabulary indices (use [UNK] or 0 for unknown words)
    ids = [vocab.get(tok, vocab.get("[UNK]", 0)) for tok in tokens]

    # Convert to tensor and add batch dimension: [1, seq_len]
    ids = torch.LongTensor(ids).unsqueeze(0).to(device)

    # Disable gradient tracking for inference
    with torch.no_grad():
        logits = model(ids).squeeze(0)          # [num_classes]
        probs = torch.softmax(logits, dim=-1)   # convert to probabilities

    # Get predicted class and its probability
    pred_class = logits.argmax().item()
    pred_prob = probs[pred_class].item()

    # Map class index to label
    sentiment = "pos" if pred_class == 1 else "neg"

    print(f"Sentiment: {sentiment} ({pred_prob:.4f})")

    return pred_class, pred_prob


In [ ]:
text = "This film is terrible!"

predict_sentiment(text, model, vocab, device)

In [ ]:
text = "This film is great!"

predict_sentiment(text, model, vocab, device)

In [ ]:
text = "This film is not terrible, it's great!"

predict_sentiment(text, model, vocab, device)

In [ ]:
text = "This film is not great, it's terrible!"

predict_sentiment(text, model, vocab, device)